In [11]:
%pip install openai newspaper3k lxml_html_clean

UsageError: %%python is a cell magic, but the cell body is empty.


In [ ]:
from lib.llm.model import (
    model_api_client,
    make_impact_from_news,
    make_reasons_from_news,
    make_operational_countries
)
from lib.scraping.scrap import extract_text_to_dataframe


In [ ]:
client = model_api_client()

In [ ]:
news_data = spark.sql("select * from hive_metastore.default.news_data WHERE Date >= DATE_SUB(CURRENT_DATE(), 7) AND Date <= CURRENT_DATE()")
stock_df = spark.sql("select * from hive_metastore.default.stock_data WHERE stockID is not NULL").dropna().reset_index(drop=True)

In [ ]:
stock_df['Countries'] = stock_df.apply(lambda row: make_operational_countries(row, client), axis=1)
stock_df = stock_df.explode('Countries')

In [ ]:
filtered_data = news_data.join(stock_df, (news_data.Actor1CountryCode == stock_df.Countries) | (news_data.Actor2CountryCode == stock_df.Countries)).toPandas()

In [ ]:
news_df_processed = extract_text_to_dataframe(filtered_data, filtered_data.columns[10], output_column = 'NewsBody')

In [ ]:
news_df_processed['impact'] = news_df_processed.apply(lambda row: make_impact_from_news(row, client), axis=1)
news_df_processed['reasons'] = news_df_processed.apply(lambda row: make_reasons_from_news(row, client), axis=1)

In [ ]:
news_df_processed.to_csv('news_data_processed.csv', index=False)

In [ ]:
spark_df_processed = spark.createDataFrame(news_df_processed)
spark_df_processed.createOrReplaceTempView("temp_view")

In [ ]:
%sql
Create or replace table hive_metastore.default.output_table_filtered AS
select Date, company_name,trading_market, industry, ActionCountryCode,impact, reasons , position, ArticleUrl   from hive_metastore.default.output_table

In [ ]:
%sql
select * from hive_metastore.default.output_table_filtered